### Import standard modules


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt

### Initialize standard logger

In [2]:
import logging

logging.basicConfig(format="%(asctime)s.%(msecs)03d %(levelname)-8s %(message)s",
                    datefmt="%Y-%m-%d %H:%M:%S",
                    level=logging.INFO)

### Create the Simulation State

There are 2 objects that hold the state of the simulation:
  1. *Population* is a container for the state each persion
  2. *Society* is the state of the government, eg the testing processes

In [3]:
from codit.society import Society
from codit.population.covid import PersonCovid
from codit.population.networks.city import CityPopulation
from codit.population.networks.city_config.city_cfg import city_paras
from codit.society.alternatives import UKSociety

In [4]:
POPULATION_SIZE = city_paras["Leeds"]['population']

In [ ]:
# Most classrooms and workplaces are closed
LOCKDOWN_CFG = dict(classrooms=0.66, workplaces=0.66, ephemeral_contact=0.0)

# We assume that Covid survivors, and the vaccinated, have an 80% chance of catching and passing on the disease another time:
X_IMMUNITY=0.8
# However this factor is square for the indian variant (and 80% squared is 64%)

pop = CityPopulation(POPULATION_SIZE, Society(config=dict(X_IMMUNITY=X_IMMUNITY)), 
                     person_type=PersonCovid, lockdown_config=LOCKDOWN_CFG)

SOCIETY_CFG = dict(DAILY_TEST_CAPACITY_PER_HEAD=0.01, 
                   PROB_APPLY_FOR_TEST_IF_SYMPTOMS=0.25)

soc = UKSociety(config=SOCIETY_CFG)

2021-05-16 15:47:02.372 INFO     Building a set of 317255 households from which to build a population


### Create the 2 variants

In [ ]:
from codit.disease import Covid

kent_variant = Covid(name="B.1.1.7")
indian_variant = Covid(name="B.1.617.2")

### Burn in the simulation

In [ ]:
from codit.outbreak import Outbreak

BURNIN_DAYS = 30
WINTER_PREVALENCE = 1/40
SEED_SIZE = int(POPULATION_SIZE * WINTER_PREVALENCE)

o_burnin = Outbreak(soc, kent_variant, n_days=BURNIN_DAYS, seed_size=SEED_SIZE, population=pop)
o_burnin.simulate()

### Run the simulation for January and February

In [ ]:
from codit.outbreak_recorder import WardComponent

DAYS_MONTH = 30

o_Jan_Feb = Outbreak(soc, n_days=DAYS_MONTH * 2, population=pop, reset_population=False)
o_Jan_Feb.recorder.add_component(WardComponent(o_Jan_Feb))
o_Jan_Feb.simulate()
o_Jan_Feb.recorder.plot(title="Jan-Feb")

### March 
  1. Open classrooms
  2. People now 'brush past' on average 0.3 others randomly per day (see COMIX)
  3. Vaccinate the over 60s

In [ ]:
LOCKDOWN_CFG = dict(classrooms=0.0, workplaces=0.66, ephemeral_contact=0.3)

pop.set_structure(soc, lockdown_config=LOCKDOWN_CFG)
pop.clear_memory()

for i, p in enumerate(pop.people):
    if p.age > 60 and i % 20: # we vaccinate 19 in every 20 people
        p.vaccinate_with("AstraZeneca")

In [ ]:
from codit.society.lateral import LateralFlowUK
lat_flow_soc = LateralFlowUK(config=SOCIETY_CFG)

In [ ]:
o_March = Outbreak(lat_flow_soc, n_days=DAYS_MONTH, population=pop, reset_population=False)
o_March.recorder.add_component(WardComponent(o_March))
o_March.simulate()
o_March.recorder.plot(title="March")

In [ ]:
o_March.recorder.components[-1].map_incidence(end_date=True)

### April
    1. More workplaces (such as non-essential retail) open up
    2. More people brush past each other as restrictions are lifted
    3. 10 people introduce the Indian variant into the Leeds population
    4. Vaccinate the over 50s

In [ ]:
LOCKDOWN_CFG = dict(classrooms=0.0, workplaces=0.5, ephemeral_contact=0.4)

pop.set_structure(lat_flow_soc, lockdown_config=LOCKDOWN_CFG)

In [ ]:
from codit.population.population import seed_infection

residents = [p for p in pop.people if 'Chapel' in p.home.ward.name]
ca, = {p.home.ward.name for p in residents}
logging.info(f"Seeding 50 infections in {ca}")
seed_infection(15, residents, indian_variant)

In [ ]:
for i, p in enumerate(pop.people):
    if (60 >= p.age > 50) and i % 20: # vaccinate 19 in every 20 people
        p.vaccinate_with("AstraZeneca")

In [ ]:
pop.clear_memory()

In [ ]:
o_April = Outbreak(lat_flow_soc, n_days=DAYS_MONTH, population=pop, reset_population=False)
o_April.recorder.add_component(WardComponent(o_April))
o_April.simulate()
o_April.recorder.plot(title="April")

In [ ]:
infected_wards = [p.home.ward for p in pop.people if p.disease == indian_variant]
print(f"{len(infected_wards)} cases of the Indian Variant:")
counts = {x.name: infected_wards.count(x) for x in infected_wards}
for k, v in counts.items():
    print(f"{v} in {k}")

In [ ]:
o_April.recorder.components[-1].map_incidence(end_date=True)

### May
    1. Even more workplaces open up
    2. More people brush past on average 0.5 others randomly per day


In [ ]:
LOCKDOWN_CFG = dict(classrooms=0.0, workplaces=0.33, ephemeral_contact=0.5)
pop.clear_memory()
pop.set_structure(lat_flow_soc, lockdown_config=LOCKDOWN_CFG)

for i, p in enumerate(pop.people):
    if (50 >= p.age > 45) and i % 20: # we vaccinate 19 in every 20 people
        p.vaccinate_with("AstraZeneca")

### Surge Vaccinations all people over 16?

In [ ]:
# for p in pop.people:
#     if p.age > 16:
#         p.vaccinate_with('AstraZeneca')

In [ ]:
o_May = Outbreak(lat_flow_soc, n_days=DAYS_MONTH, population=pop, reset_population=False)
o_May.recorder.add_component(WardComponent(o_May))
o_May.simulate()
o_May.recorder.plot(title="May")

In [ ]:
infected_wards = [p.home.ward for p in pop.people if p.disease == indian_variant]
print(f"{len(infected_wards)} cases of the Indian Variant:")
counts = {x.name: infected_wards.count(x) for x in infected_wards}
for k, v in counts.items():
    print(f"{v} in {k}")

In [ ]:
o_May.recorder.components[-1].map_incidence(end_date=True)